# Rendement Risques

In [1]:
import pandas as pd
import numpy as np

from scipy.optimize import newton
#fonction newton scipy


In [2]:
df1 = pd.DataFrame(
{
    "Coupon": [0, 0, 0, 0.035, 0.04]
    , "Maturite": [6, 9, 12, 24, 36]
    , "Prix": [99.05, 98.45, 97.85, 101.4, 102.2]
})
df1["Maturite"]= df1["Maturite"]/12
df1

,Coupon,Maturite,Prix
0,0.000,0.50,99.05
1,0.000,0.75,98.45
2,0.000,1.00,97.85
3,0.035,2.00,101.40
4,0.040,3.00,102.20


In [3]:
df2= pd.DataFrame(
{
    "Coupon": [0, 0, 0, 0.0325, 0, 0, 0]
    , "Maturite": [3, 6, 12, 24, 36, 60, 72]
    , "Prix": [100.616, 100.362, 100.18, 108.294, 101.792, 102.909, 102.472]
})
df2["Maturite"]= df2["Maturite"]/12
df2

,Coupon,Maturite,Prix
0,0.0000,0.25,100.616
1,0.0000,0.50,100.362
2,0.0000,1.00,100.180
3,0.0325,2.00,108.294
4,0.0000,3.00,101.792
5,0.0000,5.00,102.909
6,0.0000,6.00,102.472


## 1).

In [4]:
#a.
def f(x, p, c, m, n=100):
    C= c*n
    somme= 0
    for i in range(1, int(m)):
        somme += C*pow(x, i)
    return p - somme - (n+C)*pow(x, m)

# def f(x, p, c, m, n=100):
#     C= c*n
#     somme= 0
#     if m <= 1 or c == 0:
#         return p - (n+C)*pow(x,m)
#     else:
#         for i in range(1, m):
#             somme += C*pow(x, i)
#         return p - somme - (n+C)*pow(x, m)


def fprime(x, p, c, m, n=100):
    C= c*n
    somme= 0
    for i in range(1, int(m)):
            somme += i*C*pow(x, i-1)
    return - somme - (n+C)*m*pow(x, m-1)

In [5]:
f(x= 2, p=0.9528, c= 0.01, m=5, n=1)

-31.6672

In [6]:
fprime(x= 2, p=0.9528, c= 0.01, m=5, n=1)

-81.28999999999999

In [7]:
#Partie Newton
def yield_to_mat(f, x0, fprime, args):
    new= newton(func= f,x0=x0, fprime=fprime, args= args)
    return (1/new) - 1

newton(func= f
      ,x0=1.5
      ,fprime= fprime
      ,args= (0.9528, 0.01, 5, 1))

yield_to_mat(f,2,fprime,(0.9528, 0.01, 5, 1))

0.02001429157313739

In [8]:
df1_cop= pd.DataFrame()
df1_cop= df1.copy()
df1_cop

,Coupon,Maturite,Prix
0,0.000,0.50,99.05
1,0.000,0.75,98.45
2,0.000,1.00,97.85
3,0.035,2.00,101.40
4,0.040,3.00,102.20


In [9]:
# df1_cop["Yield"]= df1_cop.apply(lambda row: yield_to_mat(f, 2, fprime
#                                                       , args=
#                                                       (row[0], row["Coupon"], row["Maturite"], 100
#                                                       )), axis= 1)
# df1_cop["Maturite"]= df1_cop["Maturite"].apply(int)


df1_cop["Yield"]= df1_cop.apply(lambda row: yield_to_mat(f, 2, fprime,
                                                        args= (row[2],row[0],row[1], 100)), axis=1)
df1_cop

,Coupon,Maturite,Prix,Yield
0,0.000,0.50,99.05,0.019274
1,0.000,0.75,98.45,0.021047
2,0.000,1.00,97.85,0.021972
3,0.035,2.00,101.40,0.027708
4,0.040,3.00,102.20,0.032190


In [10]:
df2_cop= pd.DataFrame()
df2_cop= df2.copy()
df2_cop

,Coupon,Maturite,Prix
0,0.0000,0.25,100.616
1,0.0000,0.50,100.362
2,0.0000,1.00,100.180
3,0.0325,2.00,108.294
4,0.0000,3.00,101.792
5,0.0000,5.00,102.909
6,0.0000,6.00,102.472


In [11]:
df2_cop["Yield"]= df2_cop.apply(lambda row: yield_to_mat(f, 2, fprime,
                                                        args= (row[2],row[0],row[1], 100))
                               , axis= 1)
df2_cop

,Coupon,Maturite,Prix,Yield
0,0.0000,0.25,100.616,-0.024265
1,0.0000,0.50,100.362,-0.007201
2,0.0000,1.00,100.180,-0.001797
3,0.0325,2.00,108.294,-0.008445
4,0.0000,3.00,101.792,-0.005903
5,0.0000,5.00,102.909,-0.005719
6,0.0000,6.00,102.472,-0.004062


In [12]:
def prime(y, c):
    if y < c:
        return "Discount"
    elif y > c:
        return "Premium"
    else:
        return "Pair"

In [13]:
df1_cop["Comparaison"]= df1_cop.apply(lambda row: prime(row[3], row[0])
                                     ,axis= 1)
df1_cop

,Coupon,Maturite,Prix,Yield,Comparaison
0,0.000,0.50,99.05,0.019274,Premium
1,0.000,0.75,98.45,0.021047,Premium
2,0.000,1.00,97.85,0.021972,Premium
3,0.035,2.00,101.40,0.027708,Discount
4,0.040,3.00,102.20,0.032190,Discount


In [14]:
df2_cop["Comparaison"]= df2_cop.apply(lambda row: prime(row[3], row[0])
                                     ,axis= 1)
df2_cop

,Coupon,Maturite,Prix,Yield,Comparaison
0,0.0000,0.25,100.616,-0.024265,Discount
1,0.0000,0.50,100.362,-0.007201,Discount
2,0.0000,1.00,100.180,-0.001797,Discount
3,0.0325,2.00,108.294,-0.008445,Discount
4,0.0000,3.00,101.792,-0.005903,Discount
5,0.0000,5.00,102.909,-0.005719,Discount
6,0.0000,6.00,102.472,-0.004062,Discount


# Gestion du risque

D'un portefeuille obligataire

In [15]:
# somme (-i*C)/(1+y)^^(i+1) +  T(N + C)/((1+y)^^(T+1))
def macaulay_duration(p, y, c, m, n=100):
    C= c*n
    somme1, somme2= 0, 0
    for i in range(1, int(m)):
        somme1 += i*C/pow(1+y, i)
        somme2 += C/pow(1+y, i)
    return (somme1 + m*(C+n)/pow(1+y,m))/(somme2 +(C+n)/pow(1+y,m))

In [16]:
macaulay_duration(0.9585, 0.0200, 0.0, 5, 1)

4.999999999999999

In [17]:
def duration(p, y, c, m, n=100):
    return -(1/(1+y))*macaulay_duration(p, y, c, m, n)*p

In [18]:
duration(0.9585, 0.02001429157313739, 0.04, 5, 1)

-4.368229122463465

In [19]:
def duration_modif(p, y, c, m, n=100):
    return duration(p, y, c, m, n)/p

In [20]:
duration_modif(0.9585, 0.02001429157313739, 0.04, 5, 1)

-4.557359543519524

In [21]:
def duration_prof(p, y, c, m, n=100):
    C= c*n
    somme= 0
    for i in range(1, int(m)):
        somme -= i*C/pow(1+y, i+1)
    return somme - m*(C+n)/pow(1+y, m+1)

In [22]:
duration_prof(0.9585, 0.02001429157313739, 0.04, 5, 1)

-4.9866533337959265

In [23]:

#CE N EST PAS A METTRE DANS LE .PY DU PROF
df1_cop["D"]= df1_cop.apply(lambda row: duration(row[2],y=row[3], c=row[0], m=row[1], n=100)
                            , axis= 1)

df1_cop["DM"]= df1_cop.apply(lambda row:\
                            duration_modif(row[2], row[3], row[0], row[1], 100)
                            ,axis= 1)

df1_cop["MC"]= df1_cop.apply(lambda row:\
                             macaulay_duration(row[2], row[3], row[0], row[1], 100) 
                            ,axis= 1)
df1_cop

,Coupon,Maturite,Prix,Yield,Comparaison,D,DM,MC
0,0.000,0.50,99.05,0.019274,Premium,-48.588495,-0.490545,0.500000
1,0.000,0.75,98.45,0.021047,Premium,-72.315481,-0.734540,0.750000
2,0.000,1.00,97.85,0.021972,Premium,-95.746225,-0.978500,1.000000
3,0.035,2.00,101.40,0.027708,Discount,-194.018546,-1.913398,1.966414
4,0.040,3.00,102.20,0.032190,Discount,-285.892331,-2.797381,2.887427


In [24]:
df2_cop["D"]= df2_cop.apply(lambda row: duration(row[2],y=row[3], c=row[0], m=row[1], n=100)
                            , axis= 1)

df2_cop["DM"]= df2_cop.apply(lambda row:\
                            duration_modif(row[2], row[3], row[0], row[1], 100)
                            ,axis= 1)

df2_cop["MC"]= df2_cop.apply(lambda row:\
                             macaulay_duration(row[2], row[3], row[0], row[1], 100) 
                            ,axis= 1)
df2_cop

,Coupon,Maturite,Prix,Yield,Comparaison,D,DM,MC
0,0.0000,0.25,100.616,-0.024265,Discount,-25.779545,-0.256217,0.250000
1,0.0000,0.50,100.362,-0.007201,Discount,-50.544968,-0.503627,0.500000
2,0.0000,1.00,100.180,-0.001797,Discount,-100.360324,-1.001800,1.000000
3,0.0325,2.00,108.294,-0.008445,Discount,-215.127153,-1.986510,1.969733
4,0.0000,3.00,101.792,-0.005903,Discount,-307.189324,-3.017814,3.000000
5,0.0000,5.00,102.909,-0.005719,Discount,-517.504385,-5.028757,5.000000
6,0.0000,6.00,102.472,-0.004062,Discount,-617.339404,-6.024469,6.000000


Soit un portefeuille composé de tous les actifs du df2 en même proportion.

La duration est un opérateur linéaire, ainsi pour mesurer la duration de notre portefeuille équipondéré, nous allons sommer les durations de toutes les obligations au prorata de leur poids (soit ici 1/7)

In [25]:
portefeuille_mc= sum(df2_cop["MC"]/df2_cop.shape[0])
portefeuille_mc

2.5313904987230877

In [26]:
portefeuille_d= sum(df2_cop["D"]/df2_cop.shape[0])
portefeuille_d

-261.977871824832

In [27]:
portefeuille_dm= sum(df2_cop["DM"]/df2_cop.shape[0])
portefeuille_dm

-2.5455992197327912

## Calcul de la sensibilité

In [28]:
def derive1(y, c, m, n=100):
    """
    """
    C= c*n
    somme= 0
    for i in range(1, int(m)):
        somme -= i*(C)/pow(1+y, i+1)
    return somme - m*(n+C)/pow(1+y, m+1)

def derive2(y, c, m, n=100):
    """
    """
    C= c*n
    somme= 0
    for i in range(1, int(m)):
        somme += i*(i+1)*(C)/pow(1+y, i+2)
    return somme + m*(m+1)*(C+n)/pow(1+y, m+2)

def sensibilite(deltar, y, c, m, n=100):
    """
    """
    return derive1(y, c, m, n)*deltar + (1/2)*derive2(y, c, m, n)*pow(deltar,2)

# def sensibilite_modif(p, deltar, y, c, m, n=100):
#     """
#     """
#     return sensibilite(y, c, m, n)/p

def sensibilite_vrai(p, y, c, m, n=100):
    return - macaulay_duration(p, y, c, m, n)/(1+y)

In [29]:
#test
print(sensibilite(0.01, -0.008445, 0.0325, 2.00))
# 0.035	2.00	101.40	0.027708
# 0.0325	2.00	108.294	-0.008445
print(sensibilite_vrai(108.294, -0.008445, 0.0325, 2.00))

-2.118891779607947
-1.9865095524900913


In [32]:
#construction df pour des maturite de 1 à 10 ans
# pour n =100
df3= pd.DataFrame(
{
    "Coupon": np.zeros(10)+0.0325,
    "Maturite": np.arange(1, 11, 1),
    "Yield": np.zeros(10)+0.025
})
df3

,Coupon,Maturite,Yield
0,0.0325,1,0.025
1,0.0325,2,0.025
2,0.0325,3,0.025
3,0.0325,4,0.025
4,0.0325,5,0.025
5,0.0325,6,0.025
6,0.0325,7,0.025
7,0.0325,8,0.025
8,0.0325,9,0.025
9,0.0325,10,0.025
